In [1]:
import tensorflow as tf

import os
import hickle as hkl
import numpy as np
import pandas as pd
import seaborn_image as isns
import rasterio as rs
%run ../src/preprocessing/indices.py
%run ../src/preprocessing/whittaker_smoother.py


In [2]:
#predict_model_path = "../models/75-composite-masterfeb9/"
predict_model_path = "../models/tf2-nov6-44-master/"
#predict_model_path = "../models/172-ttc-dec2023-3/"
#predict_model_path = "../models/tf2-new/"



In [3]:
import sys
#!{sys.executable} -m pip install seaborn

In [4]:
predict_graph_def = tf.compat.v1.GraphDef()
if os.path.exists(predict_model_path):
    print(f"Loading model from {predict_model_path}")
    predict_file = tf.io.gfile.GFile(predict_model_path + "predict_graph-76.pb", 'rb')
    predict_graph_def.ParseFromString(predict_file.read())
    predict_graph = tf.import_graph_def(predict_graph_def, name='predict')
    predict_sess = tf.compat.v1.Session(graph=predict_graph)
    predict_logits = predict_sess.graph.get_tensor_by_name(f"conv2d/Sigmoid:0") 
    #predict_logits = predict_sess.graph.get_tensor_by_name(f"predict/clip_by_value:0") 
    #feature_extraction = predict_sess.graph.get_tensor_by_name(f"predict/csse_out_mul/mul:0")  
    predict_inp = predict_sess.graph.get_tensor_by_name("Placeholder:0")
    predict_length = predict_sess.graph.get_tensor_by_name("PlaceholderWithDefault:0")
else:
    raise Exception(f"The model path {predict_model_path} does not exist")

Loading model from ../models/tf2-nov6-44-master/
Metal device set to: Apple M3 Max


2024-02-26 09:14:49.778339: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-26 09:14:49.778496: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
SIZE = 76
min_all = [0.006576638437476157, 0.0162050812542916, 0.010040436408026246, 
               0.013351644159609368, 0.01965362020294499, 0.014229037918669413, 
               0.015289539940489814, 0.011993591210803388, 0.008239871824216068,
               0.006546120393682765, 0.0, 0.0, 0.0, -0.1409399364817101,
               -0.4973397113668104, -0.09731556326714398, -0.7193834232943873]

max_all = [0.2691233691920348, 0.3740291447318227, 0.5171435111009385, 
           0.6027466239414053, 0.5650263218127718, 0.5747005416952773,
           0.5933928435187305, 0.6034943160143434, 0.7472037842374304,
           0.7000076295109483, 
           0.4,
           0.948334642387533, 
           0.6729257769285485, 0.8177635298774327, 0.35768999002433816,
           0.7545951919107605, 0.7602693339366691]

In [6]:

def convert_to_db(x: np.ndarray, min_db: int) -> np.ndarray:
    """ Converts unitless backscatter coefficient
        to db with a min_db lower threshold
        
        Parameters:
         x (np.ndarray): unitless backscatter (T, X, Y, B) array
         min_db (int): integer from -50 to 0
    
        Returns:
         x (np.ndarray): db backscatter (T, X, Y, B) array
    """
    
    x = 10 * np.log10(x + 1/65535)
    x[x < -min_db] = -min_db
    x = (x + min_db) / min_db
    return np.clip(x, 0, 1)

In [7]:
idx = 0
pd.set_option('display.float_format', lambda x: '%.5f' % x)

def preprocess_sample(sample, idx):

    med = np.median(sample, axis = 0)
    med = med[np.newaxis, :, :, :]
   # sample = np.concatenate([sample, med], axis = 0)
    
    sample = np.core.umath.clip(sample, min_all, max_all)
    sample = (sample - midrange) / (rng / 2)
    """
    for band in range(0, sample.shape[-1]):
        mins = min_all[band]
        maxs = max_all[band]
        sample[..., band] = np.clip(sample[..., band], mins, maxs)
        midrange = (maxs + mins) / 2
        rng = maxs - mins
        standardized = (sample[..., band] - midrange) / (rng / 2)
        sample[..., band] = standardized
    """
    return sample


def predict_subtile(subtile: np.ndarray, sess: "tf.Sess", op: "tf.Tensor", size: "int") -> np.ndarray:
    """ Runs temporal (convGRU + UNET) predictions on a (12, 216, 216, 17) array:
        - Calculates remote sensing indices
        - Normalizes data
        - Returns predictions for subtile

        Parameters:
         subtile (np.ndarray): monthly sentinel 2 + sentinel 1 mosaics
         sess (tf.Session): tensorflow session for prediction
    
        Returns:
         preds (np.ndarray): (160, 160) float32 [0, 1] predictions
    """
    #np.save('subtile.npy', subtile)
    if np.sum(subtile) != 0:
        if not isinstance(subtile.flat[0], np.floating):
            print("CONVERTING TO FLOAT")
            assert np.max(subtile) > 1
            subtile = subtile / 65535.

        time1 = time.time()
        #subtile = np.core.umath.clip(subtile, min_all, max_all)
        #subtile = (subtile - midrange) / (rng / 2)
        batch_x = subtile[np.newaxis].astype(np.float32)
        lengths = np.full((batch_x.shape[0]), args.length)
        #print("length is ", LEN)
        time2 = time.time()
        
        time1 = time.time()
        #preds = sess.run(predict_earlyfeats,
       #                       feed_dict={predict_inp:batch_x, 
        #                                 predict_length:lengths})
        #batch_x = np.delete(batch_x, [11, 12], axis = -1)
        preds = sess.run(op,
                              feed_dict={predict_inp:batch_x, 
                                         predict_length:lengths})

        #preds = sess.run(op,
        #                      feed_dict={predict_inp:batch_x, 
        #                                 predict_length:lengths})


        preds = preds.squeeze()

        clip = (preds.shape[0] - size) // 2
        #preds = preds[1:-1, 1:-1]
        if clip > 0:
            preds = preds[clip:-clip, clip:-clip]
        #preds = np.clip(preds, 0, 1)
        preds = np.float32(preds)
        time2 = time.time()

    else:
        preds = np.full((SIZE, SIZE), 255)
        print(f"The sum of the subtile is {np.sum(subtile)}")
    
    return preds

In [8]:
x = np.load("../src/train-ard-y/20520248.npy")
np.max(x)

0.0

In [9]:
fpath = '/Volumes/John/train-ard-128/'
ypath = '/Volumes/John/data/train-17k-may2023/train-y/'

#fpath = '/Volumes/John/data/output-large-ard-x/'
#ypath = '/Volumes/John/data/output-large-ard-y/'
x_files = [x[:-4] for x in os.listdir(fpath) if x[-4:] == '.hkl']
train_bad = [x[:-4] for x in os.listdir('/Volumes/John/data/train-17k-may2023/bad/')]
x_files = [x for x in x_files  if x not in train_bad]
#x_files = [x for x in x_files if x[:4] == '5969']

In [10]:
x = hkl.load("/Volumes/John/data/output-large-ard-x/228876305.hkl")
print(np.mean(x, axis = (0, 1, 2)), x.shape)

[0.04386421 0.06575633 0.07844093 0.2223032  0.11906344 0.18196905
 0.20939553 0.24083103 0.24951214 0.16221696 0.04650502 0.60427976
 0.32973275 0.27134252 0.07327386 0.2455598  0.24252762] (12, 128, 128, 17)


In [11]:
def unpreprocess(x):
    x[..., -1] += 0.7193834232943873
    x[..., -1] /= 2
    
    
    #out[-1] -= 0.7193834232943873
    x[..., -2] += 0.09731556326714398
    x[..., -3] += 0.4973397113668104,
    x[..., -4] += 0.1409399364817101
    return x

In [12]:

def normalize_subtile(subtile):
    for band in range(0, subtile.shape[-1]):
        mins = min_all[band]
        maxs = max_all[band]
        subtile[..., band] = np.clip(subtile[..., band], mins, maxs)
        midrange = (maxs + mins) / 2
        rng = maxs - mins
        standardized = (subtile[..., band] - midrange) / (rng / 2)
        subtile[..., band] = standardized
    return subtile

def denormalize_subtile(subtile):
    for band in range(0, subtile.shape[-1]):
        mins = min_all[band]
        maxs = max_all[band]
        midrange = (maxs + mins) / 2
        rng = maxs - mins
        standardized = subtile[..., band] * (rng / 2)
        standardized = standardized + midrange
        subtile[..., band] = standardized
    return subtile
    
    
def make_and_smooth_indices(arr):
    """Calculates remote sensing indices
    (evi, bi, msavi2, grndvi) and smooths them
    with the Whittaker smoother
    """
    def _make_indices(arr):
        indices = np.zeros(
            (arr.shape[0], arr.shape[1], arr.shape[2], 4), dtype = np.float32
        )
        indices[:, ..., 0] = evi(arr)
        indices[:, ...,  1] = bi(arr)
        indices[:, ...,  2] = msavi2(arr)
        indices[:, ...,  3] = grndvi(arr)
        return indices

    sm_indices = Smoother(lmbd = 50, 
                          size = 12, 
                          nbands = 4, 
                          dimx = arr.shape[1],
                          dimy = arr.shape[2], 
                          outsize = 12)

    indices = _make_indices(arr)
    indices = sm_indices.interpolate_array(indices)
    return indices


def load_individual_sample(fpath, ypath, f):
    ishkl = os.path.exists(fpath + f + '.hkl')
    if ishkl:
        x = hkl.load(fpath + f + '.hkl')
        if np.max(x) > 3:
            x = x / 65535
        
    else:
        x = np.load(fpath + f + ".npy")
        if np.max(x) > 3:
            x = x / 65535
    preprocess = True
    if not preprocess:
        x = unpreprocess(x)
        x = np.clip(x, 0, 1)
    if x.shape[-1] == 13:
        i = make_and_smooth_indices(x)
        out = np.zeros((x.shape[0], x.shape[1], x.shape[2], 17), dtype = np.float32)
        out[..., :13] = x 
        out[..., 13:] = i
    else:
        out = x
        out[..., -1] *= 2
        out[..., -1] -= 0.7193834232943873
        
        out[..., -2] -= 0.09731556326714398
        out[..., -3] -= 0.4973397113668104,
        out[..., -4] -= 0.1409399364817101
            #out[]

    median = np.median(out, axis = 0)
    out = np.reshape(out, (4, 3, out.shape[1], out.shape[2], out.shape[3]))
    out = np.median(out, axis = 1, overwrite_input = True)
    out = np.concatenate([out, median[np.newaxis]], axis = 0)
    if os.path.exists(ypath + f + '.tif'):
        y = rs.open(ypath + f + '.tif').read(1)
    elif os.path.exists(ypath + f + '.npy'):
        y = np.load(ypath + f + '.npy')
    else:
        print(f"{f} Y does not exist")
        y = np.zeros((14, 14))
    if np.max(y) < 10:
        y = y * 255
    if np.max(y) > 1:
        y = y / 255

    if os.path.exists(fpath + f + ".tif"):
        img = rs.open(fpath + f + ".tif").read()
        img = np.moveaxis(img, 0, 2)
    else:
        img = np.zeros_like(y)
    #return out[:, 2:-2, 2:-2, :], y, img
    if out.shape[1] >= SIZE:
        bord_clip = (out.shape[1]-  SIZE) // 2
        if bord_clip > 0:
            return normalize_subtile(out[:, bord_clip:-bord_clip, bord_clip:-bord_clip :]), y, img
        else:
            return normalize_subtile(out), y, img
    else:
        pad = (SIZE - out.shape[1]) // 2
        out = np.pad(out, ((0, 0), (pad, pad), (pad, pad), (0, 0)), mode='reflect')
        return normalize_subtile(out), y, img
        
#x_files = ['5969114442']
x, y, img = load_individual_sample(fpath, ypath, '228876305')


In [13]:
## LEN = 4
from tqdm import tnrange
#x_batch_test = augment_sample(x[0])

import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
%matplotlib inline

best_xs = []
best_ys = []
names = []
output_size = SIZE - 14
label_size = 14
start = (output_size - label_size) // 2
plot = False
georeference = True

print(f"Plot: {plot}, Georeference: {georeference}, {len(x_files)} files")
for i in tnrange(len(x_files)):
#for i, val in test_data.iterrows():
    if os.path.exists(f"/Volumes/John/data/train-17k-may2023/figs/{x_files[i]}.png"):
        try:
            #fpath = '/Volumes/John/data/output-large-ard-x/'
            batch_x, y, img = load_individual_sample(fpath, ypath, x_files[i])
            batch_x = batch_x[np.newaxis]
            lengths = np.full((batch_x.shape[0]), 4)
            if np.prod(batch_x.shape[2:4]) == (76*76):
                preds = predict_sess.run(predict_logits,
                                      feed_dict={predict_inp:batch_x, 
                                                 predict_length:lengths})
                pred_tc = np.mean(preds.squeeze()[start:start+label_size, start:start + label_size])
                lab_tc = np.mean(y)
                if georeference:
                    best_iou = 0.
                    best_x = 0.
                    best_y = 0.
                    iter = 0
                    for xs in [0, -1, 1]:
                        for ys in [0, -1, 1]:
                            pred_tc = preds.squeeze()[start+xs:start+label_size+xs, start+ys:start + label_size+ys]
                            intersection = pred_tc * y
                            union = np.maximum(pred_tc, y)
                            iou = np.mean(intersection) / np.mean(union)
                            if iter == 0:
                                middle_iou = iou
                            if iou > (best_iou + 0.005) and iou > (middle_iou + 0.02):
                                best_iou = iou
                                best_x = xs
                                best_y = ys
                            iter += 1
                    best_xs.append(best_x)
                    best_ys.append(best_y)
                    names.append(x_files[i])
                if plot and abs(pred_tc - lab_tc) >= 0.0:
                    fig, axs = plt.subplots(2, 2, figsize = (9, 9))
                    sns.heatmap(preds.squeeze(), vmin = 0.0, vmax = 1, ax=axs[0, 1], cbar = False)
                    sns.heatmap(preds.squeeze()[start:start+14, start:start+14], vmin = 0.0, vmax = 1, ax=axs[1, 0], cbar = False)
                    sns.heatmap(y, vmin = 0.0, vmax = 1, ax=axs[1, 1], cbar = False)
                    isns.imgplot(img, ax = axs[0, 0])
                    axs[0,0].set_title(f'{i}, model {np.mean(preds)}')
                    #ax2.set_title(f'136, {np.mean(preds2)}')
                    axs[1, 0].set_title(np.mean(preds.squeeze()[start:start+14, start:start+14]))
                    axs[1, 1].set_title(np.mean(y))
                    #ax4.set_title(np.mean(test_y[i]))
                    #plt.show()
                    
                    plt.savefig(f"/Volumes/John/data/train-17k-may2023/figs/{x_files[i]}")
                    plt.close()
            else:
                print(f"Skipping {i}, {batch_x.shape}")
        #to_keep.append(i)
        except KeyboardInterrupt:
                print('Interrupted')
                try:
                    sys.exit(130)
                except SystemExit:
                    os._exit(130)
        except Exception as e:
            print(f"{i}, {x_files[i]}, Ran into {str(e)}")


Plot: {plot}, Georeference: {georeference}, {len(x_files)} files


/var/folders/kk/09dbtcq15sb84xb9mrj4g1s00000gp/T/ipykernel_50705/622740918.py:21: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(len(x_files)):


  0%|          | 0/20874 [00:00<?, ?it/s]

2024-02-26 09:14:50.682169: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2024-02-26 09:14:50.686842: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-02-26 09:14:50.687075: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/4e1473ee-9f66-11ee-8daf-cedaeb4cabe2/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x1x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/4e1473ee-9f66-11ee-8daf-cedaeb4cabe2/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mp

1, 228876300, Ran into operands could not be broadcast together with shapes (14,14) (68,68) 
157, 230967906, Ran into operands could not be broadcast together with shapes (14,14) (68,68) 
1137, 230967907, Ran into operands could not be broadcast together with shapes (14,14) (68,68) 
1159, 20920241, Ran into operands could not be broadcast together with shapes (14,14) (76,76) 
1297, 228876301, Ran into operands could not be broadcast together with shapes (14,14) (68,68) 
1337, 228876303, Ran into operands could not be broadcast together with shapes (14,14) (68,68) 
1448, 230967905, Ran into operands could not be broadcast together with shapes (14,14) (68,68) 
1450, 20820243, Ran into operands could not be broadcast together with shapes (14,14) (48,48) 
1741, 2172432018, Ran into operands could not be broadcast together with shapes (14,14) (76,76) 
1829, 1638108804, Ran into operands could not be broadcast together with shapes (14,14) (68,68) 
2109, 1638108805, Ran into operands could no

In [19]:
dfout = pd.DataFrame({'name':names, 'bestx':best_xs, 'besty':best_ys})
dfout.head(10)
dfout.to_csv("georeferencer_20k.csv", index = False)

import rasterio as rs
with rs.open('../../HighResCanopyHeight/data/images2/ppcclip.tif') as f:
    profile = f.profile.copy()
    transform = f.bounds
    
    data = f.read()
    data = data.reshape(3, 1566 // 2, 2, 1482 // 2, 2)
    data = np.mean(data, axis = (2, 4))
    data = data.astype(np.uint8)
    newtransform = rs.transform.from_bounds(f.bounds[0], f.bounds[1], f.bounds[2], f.bounds[3], data.shape[2], data.shape[1])
    print(data.shape, np.max(data), data.dtype)
    # And then change the band count to 1, set the
    # dtype to uint8, and specify LZW compression.
    profile.update(
        dtype=rs.uint8,
        count=3,
        height = data.shape[1],
        width = data.shape[2],
        transform = newtransform,
        compress='lzw')
    print(profile)

    with rs.open('example.tif', 'w', **profile) as dst:
        dst.write(data.astype(rs.uint8))


In [15]:
#xs = np.load("train_xs.npy")
#train_xs = [x[:-4] for x in os.listdir("/Volumes/John/data/train-17k-may2023/figs/") if x[-4:] == '.png']
#train_xs = [x for x in train_xs  if x not in train_bad]
#print(len(train_xs))
#np.save("train_xs.npy", train_xs)

In [16]:
#train_xs = [x[:-4] for x in os.listdir("/Volumes/John/data/train-17k-may2023/remove/") if x[-4:] == '.png']

In [17]:
#np.array(train_xs)

In [18]:
#len(train_xs)